<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Testing" data-toc-modified-id="Testing-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Testing</a></span><ul class="toc-item"><li><span><a href="#The-go-test-Tool" data-toc-modified-id="The-go-test-Tool-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>The go test Tool</a></span></li><li><span><a href="#Test-Functions" data-toc-modified-id="Test-Functions-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Test Functions</a></span><ul class="toc-item"><li><span><a href="#Randomized-Testing" data-toc-modified-id="Randomized-Testing-11.2.1"><span class="toc-item-num">11.2.1&nbsp;&nbsp;</span>Randomized Testing</a></span></li><li><span><a href="#Testing-a-Command" data-toc-modified-id="Testing-a-Command-11.2.2"><span class="toc-item-num">11.2.2&nbsp;&nbsp;</span>Testing a Command</a></span></li><li><span><a href="#White-Box-Testing" data-toc-modified-id="White-Box-Testing-11.2.3"><span class="toc-item-num">11.2.3&nbsp;&nbsp;</span>White-Box Testing</a></span></li></ul></li><li><span><a href="#Coverage" data-toc-modified-id="Coverage-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Coverage</a></span></li><li><span><a href="#Benchmark-Functions" data-toc-modified-id="Benchmark-Functions-11.4"><span class="toc-item-num">11.4&nbsp;&nbsp;</span>Benchmark Functions</a></span></li><li><span><a href="#Profiling" data-toc-modified-id="Profiling-11.5"><span class="toc-item-num">11.5&nbsp;&nbsp;</span>Profiling</a></span></li><li><span><a href="#Example-Functions" data-toc-modified-id="Example-Functions-11.6"><span class="toc-item-num">11.6&nbsp;&nbsp;</span>Example Functions</a></span></li></ul></li></ul></div>

# Testing

## The go test Tool

## Test Functions

In [1]:
import "unicode"
func IsPalindrome(s string) bool {
	var letters []rune
	for _, r := range s {
		if unicode.IsLetter(r) {
			letters = append(letters, unicode.ToLower(r))
		}
	}
	for i := range letters {
		if letters[i] != letters[len(letters)-1-i] {
			return false
		}
	}
	return true
}

In [34]:
package word

import "testing"

func TestPalindrome(t *testing.T) {
        if !IsPalindrome("detartrated") {
                t.Error(`IsPalindrome("detartrated") = false`)
        }
        if !IsPalindrome("kayak") {
                t.Error(`IsPalindrome("kayak") = false`)
        }
}

func TestNonPalindrome(t *testing.T) {
        if IsPalindrome("palindrome") {
                t.Error(`IsPalindrome("palindrome") = true`)
        }
}

<code>
$ go test $GOPATH/src/gopl.io/ch11/word1
</code>

In [39]:
// IsPalindrome reports whether s reads the same forward and backward.
// Letter case is ignored, as are non-letters.
func IsPalindrome(s string) bool {
        //var letters []rune
        letters := make([]rune,0, len(s))
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        //for i := range letters {
        n := len(letters)/2
        for i :=0; i<n; i++ {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}


In [41]:
import (
        "fmt"
        "math/rand"
        "time"
)

//!+bench

import "testing"

//!-bench

//!+test
func TestIsPalindrome(t *testing.T) {
        var tests = []struct {
                input string
                want  bool
        }{
                {"", true},
                {"a", true},
                {"aa", true},
                {"ab", false},
                {"kayak", true},
                   {"detartrated", true},
                {"A man, a plan, a canal: Panama", true},
                {"Evil I did dwell; lewd did I live.", true},
                {"Able was I ere I saw Elba", true},
                {"été", true},
                {"Et se resservir, ivresse reste.", true},
                {"palindrome", false}, // non-palindrome
                {"desserts", false},   // semi-palindrome
        }
        for _, test := range tests {
                if got := IsPalindrome(test.input); got != test.want {
                        t.Errorf("IsPalindrome(%q) = %v", test.input, got)
                }
        }
}


### Randomized Testing

In [42]:
import "math/rand"
func randomPalindrome(rng *rand.Rand) string {
	n := rng.Intn(25) // random length up to 24
	runes := make([]rune, n)
	for i := 0; i < (n+1)/2; i++ {
		r := rune(rng.Intn(0x1000)) // random rune up to '\u0999'
		runes[i] = r
		runes[n-1-i] = r
	}
	return string(runes)
}

In [43]:
import "testing"
import "time"
func TestRandomPalindromes(t *testing.T) {
	// Initialize a pseudo-random number generator.
	seed := time.Now().UTC().UnixNano()
	t.Logf("Random seed: %d", seed)
	rng := rand.New(rand.NewSource(seed))

	for i := 0; i < 1000; i++ {
		p := randomPalindrome(rng)
		if !IsPalindrome(p) {
			t.Errorf("IsPalindrome(%q) = false", p)
		}
	}
}

### Testing a Command

In [44]:
//gopl.io/ch11/echo
import (
	"flag"
	"fmt"
	"io"
	"os"
	"strings"
)

var (
	n = flag.Bool("n", false, "omit trailing newline")
	s = flag.String("s", " ", "separator")
)
var out io.Writer = os.Stdout // modified during testing

func main() {
	flag.Parse()
	if err := echo(!*n, *s, flag.Args()); err != nil {
		fmt.Fprintf(os.Stderr, "echo: %v\n", err)
		os.Exit(1)
	}
}

func echo(newline bool, sep string, args []string) error {
	fmt.Fprint(out, strings.Join(args, sep))
	if newline {
		fmt.Fprintln(out)
	}
	return nil
}

In [45]:
// Test of echo command.  Run with: go test gopl.io/ch11/echo

//!+
//package main

import (
	"bytes"
	"fmt"
	"testing"
)

func TestEcho(t *testing.T) {
	var tests = []struct {
		newline bool
		sep     string
		args    []string
		want    string
	}{
		{true, "", []string{}, "\n"},
		{false, "", []string{}, ""},
		{true, "\t", []string{"one", "two", "three"}, "one\ttwo\tthree\n"},
		{true, ",", []string{"a", "b", "c"}, "a,b,c\n"},
		{false, ":", []string{"1", "2", "3"}, "1:2:3"},
	}

	for _, test := range tests {
		descr := fmt.Sprintf("echo(%v, %q, %q)",
			test.newline, test.sep, test.args)

		out = new(bytes.Buffer) // captured output
		if err := echo(test.newline, test.sep, test.args); err != nil {
			t.Errorf("%s failed: %v", descr, err)
			continue
		}
		got := out.(*bytes.Buffer).String()
		if got != test.want {
			t.Errorf("%s = %q, want %q", descr, got, test.want)
		}
	}
}


### White-Box Testing

In [20]:
//gopl.io/ch11/storage1
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 311.

// Package storage is part of a hypothetical cloud storage server.
//!+main
//package storage

import (
	"fmt"
	"log"
	"net/smtp"
)

var usage = make(map[string]int64)

func bytesInUse(username string) int64 { return usage[username] }

// Email sender configuration.
// NOTE: never put passwords in source code!
const sender = "notifications@example.com"
const password = "correcthorsebatterystaple"
const hostname = "smtp.example.com"

const template = `Warning: you are using %d bytes of storage,
%d%% of your quota.`

func CheckQuota(username string) {
	used := bytesInUse(username)
	const quota = 1000000000 // 1GB
	percent := 100 * used / quota
	if percent < 90 {
		return // OK
	}
	msg := fmt.Sprintf(template, used, percent)
	auth := smtp.PlainAuth("", sender, password, hostname)
	err := smtp.SendMail(hostname+":587", auth, sender,
		[]string{username}, []byte(msg))
	if err != nil {
		log.Printf("smtp.SendMail(%s) failed: %s", username, err)
	}
}

//!-main

In [28]:
//gopl.io/ch11/storage2
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 312.

// Package storage is part of a hypothetical cloud storage server.
//package storage

import (
	"fmt"
	"log"
	"net/smtp"
)

var usage = make(map[string]int64)

func bytesInUse(username string) int64 { return usage[username] }

// E-mail sender configuration.
// NOTE: never put passwords in source code!
const sender = "notifications@example.com"
const password = "correcthorsebatterystaple"
const hostname = "smtp.example.com"

const template = `Warning: you are using %d bytes of storage,
%d%% of your quota.`

//!+factored
var notifyUser = func(username, msg string) {
	auth := smtp.PlainAuth("", sender, password, hostname)
	err := smtp.SendMail(hostname+":587", auth, sender,
		[]string{username}, []byte(msg))
	if err != nil {
		log.Printf("smtp.SendMail(%s) failed: %s", username, err)
	}
}

func CheckQuota(username string) {
	used := bytesInUse(username)
	const quota = 1000000000 // 1GB
	percent := 100 * used / quota
	if percent < 90 {
		return // OK
	}
	msg := fmt.Sprintf(template, used, percent)
	notifyUser(username, msg)
}

//!-factored

In [29]:
//gopl.io/ch11/quota_test.go
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

//!+test
//package storage

import (
	"strings"
	"testing"
)

func TestCheckQuotaNotifiesUser(t *testing.T) {
	var notifiedUser, notifiedMsg string
	notifyUser = func(user, msg string) {
		notifiedUser, notifiedMsg = user, msg
	}

	const user = "joe@example.org"
	usage[user] = 980000000 // simulate a 980MB-used condition

	CheckQuota(user)
	if notifiedUser == "" && notifiedMsg == "" {
		t.Fatalf("notifyUser not called")
	}
	if notifiedUser != user {
		t.Errorf("wrong user (%s) notified, want %s",
			notifiedUser, user)
	}
	const wantSubstring = "98% of your quota"
	if !strings.Contains(notifiedMsg, wantSubstring) {
		t.Errorf("unexpected notification message <<%s>>, "+
			"want substring %q", notifiedMsg, wantSubstring)
	}
}

//!-test

/*
//!+defer
func TestCheckQuotaNotifiesUser(t *testing.T) {
	// Save and restore original notifyUser.
	saved := notifyUser
	defer func() { notifyUser = saved }()
	// Install the test's fake notifyUser.
	var notifiedUser, notifiedMsg string
	notifyUser = func(user, msg string) {
		notifiedUser, notifiedMsg = user, msg
	}
	// ...rest of test...
}
//!-defer
*/

## Coverage

In [47]:
import ("fmt";"math";"testing")
func TestEval(t *testing.T) {
        tests := []struct {
                expr string
                env  Env
                want string
        }{
                {"sqrt(A / pi)", Env{"A": 87616, "pi": math.Pi}, "167"},
                {"pow(x, 3) + pow(y, 3)", Env{"x": 12, "y": 1}, "1729"},
                {"pow(x, 3) + pow(y, 3)", Env{"x": 9, "y": 10}, "1729"},
                {"5 / 9 * (F - 32)", Env{"F": -40}, "-40"},
                {"5 / 9 * (F - 32)", Env{"F": 32}, "0"},
                {"5 / 9 * (F - 32)", Env{"F": 212}, "100"},
                //!-Eval
                // additional tests that don't appear in the book
                {"-1 + -x", Env{"x": 1}, "-2"},
                {"-1 - x", Env{"x": 1}, "-2"},
                //!+Eval
        }
        var prevExpr string
        for _, test := range tests {
                // Print expr only when it changes.
                if test.expr != prevExpr {
                        fmt.Printf("\n%s\n", test.expr)
                        prevExpr = test.expr
                }
                expr, err := Parse(test.expr)
                if err != nil {
                        t.Error(err) // parse error
                        continue
                }
                got := fmt.Sprintf("%.6g", expr.Eval(test.env))
                fmt.Printf("\t%v => %s\n", test.env, got)
                if got != test.want {
                        t.Errorf("%s.Eval() in %v = %q, want %q\n",
                                test.expr, test.env, got, test.want)
                }
        }
}

ERROR: repl.go:5:22: undefined identifier: Env

<code>
$ go test -v -run=Coverage gopl.io/ch7/eval 
=== RUN   TestCoverage
--- PASS: TestCoverage (0.00s)
PASS
ok      gopl.io/ch7/eval    0.011s
</code>

<code>
$ go test -v -run=Coverage -coverprofile=c.out gopl.io/ch7/eval
=== RUN   TestCoverage
--- PASS: TestCoverage (0.00s)
PASS
coverage: 63.8% of statements
ok      gopl.io/ch7/eval        0.018s  coverage: 63.8% of statements
</code>

<code>
$ go tool cover -html=c.out
HTML output written to /tmp/cover854632247/coverage.html
</code>

[coverage.html](cover120216535/coverage.html)

## Benchmark Functions

In [49]:
func BenchmarkIsPalindrome(b *testing.B) {
        for i := 0; i < b.N; i++ {
                IsPalindrome("A man, a plan, a canal: Panama")
        }
}

In [50]:
//word.go.1
func IsPalindrome(s string) bool {
        var letters []rune
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        for i := range letters {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}

<code>
$ rm word.go; ln -s word.go.1 word.go; go test -bench=.
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          3274288               376.9 ns/op
PASS
ok      gopl.io/ch11/word2      1.627s
</code>

<code>
rm word.go; ln -s word.go.1 word.go; go test -bench=. -benchmem
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          2169936               545.5 ns/op           248 B/op          5 allocs/op
PASS
ok      gopl.io/ch11/word2      1.773s
</code>

-----------

In [51]:
//word.go.2
func IsPalindrome(s string) bool {
        var letters []rune
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        //for i := range letters {
        n := len(letters)/2
        for i :=0; i<n; i++ {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}

<code>
$ rm word.go; ln -s word.go.2 word.go; go test -bench=.
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          3230330               363.3 ns/op
PASS
ok      gopl.io/ch11/word2      1.571s
</code>

<code>
rm word.go; ln -s word.go.2 word.go; go test -bench=. -benchmem
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          3204992               364.9 ns/op           248 B/op          5 allocs/op
PASS
ok      gopl.io/ch11/word2      1.574s
    </code>

----

In [52]:
//word.go.3
func IsPalindrome(s string) bool {
        //var letters []rune
        letters := make([]rune,0, len(s))
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        for i := range letters {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}

<code>
$ rm word.go; ln -s word.go.3 word.go; go test -bench=.
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          5476538               209.7 ns/op
PASS
ok      gopl.io/ch11/word2      1.390s
</code>

<code>
rm word.go; ln -s word.go.3 word.go; go test -bench=. -benchmem
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          5408686               211.7 ns/op           128 B/op          1 allocs/op
PASS
ok      gopl.io/ch11/word2      1.390s
</code>

------

In [53]:
//word.go.4
func IsPalindrome(s string) bool {
        //var letters []rune
        letters := make([]rune,0, len(s))
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        //for i := range letters {
        n := len(letters)/2
        for i :=0; i<n; i++ {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}

<code>
rm word.go; ln -s word.go.4 word.go; go test -bench=.
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          5644834               191.5 ns/op
PASS
ok      gopl.io/ch11/word2      1.319s
</code>

<code>
rm word.go; ln -s word.go.4 word.go; go test -bench=. -benchmem
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          5759314               194.2 ns/op           128 B/op          1 allocs/op
PASS
ok      gopl.io/ch11/word2      1.354s
</code>

## Profiling

<code>
$ ulimit -n 4000
$ go test -run=NONE -bench=ClientServerParallelTLS64 -cpuprofile=cpu.log net/http
PASS
goos: linux
goarch: amd64
pkg: net/http
cpu: Intel(R) Xeon(R) CPU           X5675  @ 3.07GHz
BenchmarkClientServerParallelTLS64-4    2021/12/21 16:37:16 http: TLS handshake error fr
om 127.0.0.1:42421: read tcp 127.0.0.1:36819->127.0.0.1:42421: use of closed network con
nection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42450: read tcp 127.0.0.1:3
6819->127.0.0.1:42450: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42463: read tcp 127.0.0.1:3
6819->127.0.0.1:42463: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42454: write tcp 127.0.0.1:
36819->127.0.0.1:42454: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42481: read tcp 127.0.0.1:3
6819->127.0.0.1:42481: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42483: read tcp 127.0.0.1:3
6819->127.0.0.1:42483: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42456: write tcp 127.0.0.1:
36819->127.0.0.1:42456: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42479: write tcp 127.0.0.1:
36819->127.0.0.1:42479: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42471: write tcp 127.0.0.1:
36819->127.0.0.1:42471: use of closed network connection
</code>

<code>
go tool pprof -text -nodecount=10 ./http.test cpu.log
File: http.test
Type: cpu
Time: Dec 21, 2021 at 4:37pm (CST)
Duration: 3.95s, Total samples = 9300ms (235.73%)
Showing nodes accounting for 6210ms, 66.77% of 9300ms total
Dropped 355 nodes (cum <= 46.50ms)
Showing top 10 nodes out of 229
      flat  flat%   sum%        cum   cum%
    1840ms 19.78% 19.78%     1840ms 19.78%  math/big.addMulVVW
    1340ms 14.41% 34.19%     1340ms 14.41%  vendor/golang.org/x/crypto/curve25519.ladderstep
    1060ms 11.40% 45.59%     1080ms 11.61%  syscall.Syscall
     560ms  6.02% 51.61%     2400ms 25.81%  math/big.nat.montgomery
     550ms  5.91% 57.53%      550ms  5.91%  crypto/sha256.block
     220ms  2.37% 59.89%      440ms  4.73%  runtime.pcvalue
     180ms  1.94% 61.83%      210ms  2.26%  runtime.step
     160ms  1.72% 63.55%      160ms  1.72%  vendor/golang.org/x/crypto/curve25519.square
     150ms  1.61% 65.16%      170ms  1.83%  runtime.findObject
     150ms  1.61% 66.77%      850ms  9.14%  runtime.mallocgc
</code>

## Example Functions

In [56]:
func ExampleIsPalindrome() {
        fmt.Println(IsPalindrome("A man, a plan, a canal: Panama"))
        fmt.Println(IsPalindrome("palindrome"))
        // Output:
        // true
        // false
}
